In [ ]:
import sys
sys.path.append("..")

import datetime as dt
import io
import os
import pandas as pd
import requests
import zipfile

from lib import bitget_loader

# K-Lines

In [ ]:
bitget_loader.download_klines_in_date_range('TONUSDT', dt.date(2022,1,1), dt.date(2024,4,30))

In [ ]:
bitget_loader.load_klines_in_date_range('SOLUSDT', dt.date(2022,1,1), dt.date(2022,1,31))

# Tick

In [ ]:
def download_ticks_on_date(symbol, d):
    n = 1
    ddfs = []
    while True:
        filestr = f"{symbol}_UMCBL_{d.strftime('%Y%m%d')}_{n:03}"
        url = f"https://img.bitgetimg.com/online/trades/UMCBL/{symbol}/{filestr}.zip"
        r = requests.get(url)
        if r.status_code != 200:
            break
        z = zipfile.ZipFile(io.BytesIO(r.content))
        z.extract(f"{filestr}.csv", f"temp")
        ddfs.append(pd.read_csv(f"temp/{filestr}.csv"))
        n += 1
    if ddfs:
        outdir = f'../data/{symbol}'
        if not os.path.exists(outdir):
            os.mkdir(outdir)
    
        pd.concat(ddfs).to_parquet(f"../data/tick/{symbol}/{d.strftime('%Y%m%d')}.pq")
    else:
        print(f"No data on {d}")
        print(url)

In [ ]:
download_ticks_on_date('SOLUSDT', dt.date(2024,4,19))